In [3]:
import pandas as pd

df = pd.read_csv('/kaggle/input/gsrgdfgdsgsd/train.csv (1).csv')

In [4]:
df

,Unnamed: 0,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,...,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортиме...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1428,6.0,"{PRICE,PROMOTIONS}",Можно немного скинуть минимальную сумму заказа...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4618,8701,26325,2.0,{PRODUCTS_QUALITY},Привезли прокисший суп,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4619,8703,661,6.0,{DELIVERY},пойдет,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4620,8704,1870,6.0,"{PROMOTIONS,PRICE,SUPPORT,PRODUCTS_QUALITY}",Не дают абузить поддержка не возвращает деньги...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4621,8705,22650,2.0,"{DELIVERY,PRODUCTS_QUALITY}","Очень плохая доставка в первую очередь, постоя...",1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
y = df[[f"trend_id_res{i}" for i in range(50)]]

In [6]:
y.apply(lambda x: x.sum(), axis = 1)

0       1
1       1
2       1
3       2
4       1
       ..
4618    1
4619    1
4620    1
4621    2
4622    1
Length: 4623, dtype: int64

In [7]:
df['input_text'] = df['tags'] + df['text']

In [8]:
df['input_text'] = df['input_text'].astype('str')

In [9]:
df['input_text']

0       {ASSORTMENT,PROMOTIONS,DELIVERY}Маленький выбо...
1       {ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}Бы...
2       {DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}...
3       {PRICE,PROMOTIONS,ASSORTMENT}Наценка и ассорти...
4       {PRICE,PROMOTIONS}Можно немного скинуть минима...
                              ...                        
4618             {PRODUCTS_QUALITY}Привезли прокисший суп
4619                                     {DELIVERY}пойдет
4620    {PROMOTIONS,PRICE,SUPPORT,PRODUCTS_QUALITY}Не ...
4621    {DELIVERY,PRODUCTS_QUALITY}Очень плохая достав...
4622    {ASSORTMENT,PRICE,PROMOTIONS,DELIVERY,CATALOG_...
Name: input_text, Length: 4623, dtype: object

In [10]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = text.strip()
    return text

df['input_text'] = df['input_text'].map(lambda x: clean_text(x))

In [11]:
df['input_text']

0       {ASSORTMENT,PROMOTIONS,DELIVERY}Маленький выбо...
1       {ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}Бы...
2       {DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}...
3       {PRICE,PROMOTIONS,ASSORTMENT}Наценка и ассорти...
4       {PRICE,PROMOTIONS}Можно немного скинуть минима...
                              ...                        
4618             {PRODUCTS_QUALITY}Привезли прокисший суп
4619                                     {DELIVERY}пойдет
4620    {PROMOTIONS,PRICE,SUPPORT,PRODUCTS_QUALITY}Не ...
4621    {DELIVERY,PRODUCTS_QUALITY}Очень плохая достав...
4622    {ASSORTMENT,PRICE,PROMOTIONS,DELIVERY,CATALOG_...
Name: input_text, Length: 4623, dtype: object

In [12]:
import numpy as np

np.unique(y.to_numpy().flatten())

array([0, 1])

In [13]:
len(y.to_numpy().flatten())

231150

In [14]:
import nltk
from nltk.corpus import stopwords

q = stopwords.words('russian')

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import pandas as pd

# Проверка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка токенизатора
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# --- 1. Класс CustomDataset ---
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Токенизация
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.float),
        }

# --- 2. Загрузка данных ---
# Пример датасета

# Разделение данных на обучение и тест
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['input_text'], y.to_numpy(), test_size=0.2, random_state=42
)

# Создание Dataset
max_len = 128
train_dataset = CustomDataset(train_texts.tolist(), train_labels.tolist(), tokenizer, max_len)
val_dataset = CustomDataset(val_texts.tolist(), val_labels.tolist(), tokenizer, max_len)

# DataLoader
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# --- 3. Создание модели ---
num_labels = 50 # Количество классов
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model.to(device)

# --- 4. Обучение модели ---
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.BCEWithLogitsLoss()

# Функция обучения
def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
    
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
    
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
    
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    
        # Получение предсказаний
        preds = torch.sigmoid(outputs.logits).detach().cpu().numpy()
        preds = (preds > 0.5).astype(int)  # Бинаризация
    
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds)

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="macro")
    return total_loss / len(dataloader), acc, f1

# Функция валидации
def eval_epoch(model, dataloader, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
    
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)
    
            total_loss += loss.item()
    
            # Получение предсказаний
            preds = torch.sigmoid(outputs.logits).detach().cpu().numpy()
            preds = (preds > 0.5).astype(int)  # Бинаризация
    
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds)

    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average="macro")
    return total_loss / len(dataloader), acc, f1

# --- 5. Цикл обучения ---
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss, train_acc, train_f1 = train_epoch(model, train_loader, optimizer, device)
    val_loss, val_acc, val_f1 = eval_epoch(model, val_loader, device)

    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, F1: {train_f1:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, F1: {val_f1:.4f}")

# --- 6. Сохранение модели ---
torch.save(model.state_dict(), '/kaggle/working/rubert_model.pth')


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

...

Train Loss: 0.0410, Accuracy: 0.5706, F1: 0.2505
Val Loss: 0.0554, Accuracy: 0.4746, F1: 0.2394
Epoch 9/10


Validation: 100%|██████████| 116/116 [00:06<00:00, 16.61it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Train Loss: 0.0354, Accuracy: 0.6230, F1: 0.2979
Val Loss: 0.0531, Accuracy: 0.4714, F1: 0.2353
Epoch 10/10


Validation: 100%|██████████| 116/116 [00:06<00:00, 16.61it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Train Loss: 0.0310, Accuracy: 0.6652, F1: 0.3350
Val Loss: 0.0521, Accuracy: 0.4962, F1: 0.2803


In [ ]:
import torch
import gc

def clear_gpu_memory():
    gc.collect()  # Очистка объектов Python
    torch.cuda.empty_cache()  # Очистка кеша CUDA
    torch.cuda.ipc_collect()  # Освобождение дополнительных ресурсов IPC
    print("GPU memory cleared.")

clear_gpu_memory()


In [17]:
df_test = pd.read_csv('/kaggle/input/fdbdfbdf/test (1).csv')

In [97]:
df_test

,Unnamed: 0,index,assessment,tags,text
0,1,3135,3.0,{DELIVERY},"Последнее время думаю плохо, сроки доставки да..."
1,3,4655,2.0,"{PRICE,DELIVERY,ASSORTMENT}",Цены намного выше магазинных но радуют акции
2,5,22118,2.0,"{CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}","Доставка за [NUM] минут, заказ даже не начали ..."
3,7,23511,0.0,{DELIVERY},Ужасно долгая доставка
4,8,45,6.0,"{ASSORTMENT,PROMOTIONS}",Добрый вечер! Вы большие молодцы. Меня всё уст...
...,...,...,...,...,...
9010,16992,3523,3.0,"{PRICE,SUPPORT,DELIVERY}",Задержка с доставкой не даете промокод на скид...
9011,16993,24925,6.0,"{PRICE,PRODUCTS_QUALITY,ASSORTMENT}",Очень удобный формат сервиса и очень маленький...
9012,16994,6327,6.0,"{PAYMENT,ASSORTMENT,DELIVERY}","Сумма заказа почти всегда высокая, что зачасту..."
9013,16997,530,3.0,"{PRODUCTS_QUALITY,SUPPORT,DELIVERY}","Часто, заказываю у вас молочную продукцию, при..."


In [18]:
df_test['input_text'] = df_test['tags'] + df_test['text']

In [19]:
df_test

,Unnamed: 0,index,assessment,tags,text,input_text
0,1,3135,3.0,{DELIVERY},"Последнее время думаю плохо, сроки доставки да...","{DELIVERY}Последнее время думаю плохо, сроки д..."
1,3,4655,2.0,"{PRICE,DELIVERY,ASSORTMENT}",Цены намного выше магазинных но радуют акции,"{PRICE,DELIVERY,ASSORTMENT}Цены намного выше м..."
2,5,22118,2.0,"{CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}","Доставка за [NUM] минут, заказ даже не начали ...","{CATALOG_NAVIGATION,ASSORTMENT,DELIVERY}Достав..."
3,7,23511,0.0,{DELIVERY},Ужасно долгая доставка,{DELIVERY}Ужасно долгая доставка
4,8,45,6.0,"{ASSORTMENT,PROMOTIONS}",Добрый вечер! Вы большие молодцы. Меня всё уст...,"{ASSORTMENT,PROMOTIONS}Добрый вечер! Вы больши..."
...,...,...,...,...,...,...
9010,16992,3523,3.0,"{PRICE,SUPPORT,DELIVERY}",Задержка с доставкой не даете промокод на скид...,"{PRICE,SUPPORT,DELIVERY}Задержка с доставкой н..."
9011,16993,24925,6.0,"{PRICE,PRODUCTS_QUALITY,ASSORTMENT}",Очень удобный формат сервиса и очень маленький...,"{PRICE,PRODUCTS_QUALITY,ASSORTMENT}Очень удобн..."
9012,16994,6327,6.0,"{PAYMENT,ASSORTMENT,DELIVERY}","Сумма заказа почти всегда высокая, что зачасту...","{PAYMENT,ASSORTMENT,DELIVERY}Сумма заказа почт..."
9013,16997,530,3.0,"{PRODUCTS_QUALITY,SUPPORT,DELIVERY}","Часто, заказываю у вас молочную продукцию, при...","{PRODUCTS_QUALITY,SUPPORT,DELIVERY}Часто, зака..."


In [20]:
df_test['input_text'] = df_test['input_text'].astype('str')

In [21]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
    text = text.strip()
    return text

df_test['input_text'] = df_test['input_text'].map(lambda x: clean_text(x))

In [22]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import pandas as pd
import numpy as np

# Проверка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка токенизатора
model_name = "DeepPavlov/rubert-base-cased"
# tokenizer = BertTokenizer.from_pretrained(model_name)

# --- 1. Класс CustomDataset ---
class CustomDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        
        # Токенизация
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        if self.labels is not None:
            label = self.labels[idx]
            return {
                "input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0),
                "label": torch.tensor(label, dtype=torch.float),
            }
        else:
            return {
                "input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0),
            }

# --- 2. Загрузка данных ---
# Пример датасета (без меток для теста)
test_texts = df_test['input_text']  # Например, данные для теста без меток

# Создание Dataset
test_dataset = CustomDataset(test_texts.tolist(), tokenizer=tokenizer, max_len=128)
test_loader = DataLoader(test_dataset, batch_size=8)

# --- 3. Создание модели ---
num_labels = 50  # Количество классов
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
# model.to(device)

# --- 4. Загрузка модели ---
# Загрузите сохраненную модель (если необходимо)
# model.load_state_dict(torch.load('/kaggle/working/rubert_model.pth'))

# --- 5. Предсказания на тестовых данных ---
def predict(model, dataloader, device):
    model.eval()
    all_preds = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
    
            # Получение предсказаний
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.sigmoid(outputs.logits).detach().cpu().numpy()
            preds = (preds > 0.5).astype(int)  # Бинаризация

            all_preds.extend(preds)

    return np.array(all_preds)

# Получаем предсказания на тестовом наборе
predictions = predict(model, test_loader, device)

# Печать или сохранение предсказаний
# Например, сохранить в DataFrame для дальнейшего анализа
predictions_df = pd.DataFrame(predictions, columns=[f"label_{i}" for i in range(num_labels)])

# Вывести предсказания
print(predictions_df.head())

# Или сохранить в файл
predictions_df.to_csv('/kaggle/working/test_predictions.csv', index=False)


Predicting: 100%|██████████| 1127/1127 [01:09<00:00, 16.29it/s]


   label_0  label_1  label_2  label_3  label_4  label_5  label_6  label_7  \
0        0        1        1        0        0        0        0        0   
1        0        0        0        0        0        0        0        0   
2        0        0        1        0        0        0        0        0   
3        1        0        0        0        0        0        0        0   
4        0        0        0        0        0        0        0        0   

   label_8  label_9  ...  label_40  label_41  label_42  label_43  label_44  \
0        0        0  ...         0         0         0         0         0   
1        0        0  ...         0         0         0         0         0   
2        0        0  ...         0         0         0         0         0   
3        0        0  ...         0         0         0         0         0   
4        0        0  ...         0         0         0         0         0   

   label_45  label_46  label_47  label_48  label_49  
0         0   

In [23]:
predictions_df

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,...,label_40,label_41,label_42,label_43,label_44,label_45,label_46,label_47,label_48,label_49
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9010,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9012,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
pred = pd.DataFrame()

pred['index'] = df_test['index']

In [26]:
pred

,index
0,3135
1,4655
2,22118
3,23511
4,45
...,...
9010,3523
9011,24925
9012,6327
9013,530


In [33]:
q =predictions_df.apply(lambda x: np.where(x == 1)[0], axis = 1)
pd.Series(q)

0         [1, 2, 12]
1               [12]
2            [2, 35]
3                [0]
4                 []
            ...     
9010             [3]
9011    [12, 23, 28]
9012             [8]
9013            [15]
9014              []
Length: 9015, dtype: object

In [38]:
c = q.map(lambda x: ' '.join([str(n) for n in x] if len(x)!=0 else ""))

In [39]:
c

0         1 2 12
1             12
2           2 35
3              0
4               
          ...   
9010           3
9011    12 23 28
9012           8
9013          15
9014            
Length: 9015, dtype: object

In [42]:
pred = pd.concat([pred,c], axis = 1)

In [46]:
pred.rename(columns = {0:'target'}, inplace = True)

In [47]:
pred.to_csv('/kaggle/working/predictions.csv')